In [ ]:
import numpy as np
from scipy.special import hankel1 as besselh
from scipy.special import iv as besseli
from scipy.special import kv as besselk
import scipy.integrate as integrate
import scipy.linalg as linalg
import matplotlib.pyplot as plt
from math import sqrt, cosh, cos, sinh, sin, pi
from scipy.optimize import newton, minimize_scalar
from multi_constants import *
from multi_equations import *
import pandas as pd

In [ ]:
boundary_count = len(NMK) - 1
for arr in [a, d, heaving]:
    assert len(arr) == boundary_count, "NMK should have one more entry than a, d, and heaving, which should all have the same number of entries."

for entry in heaving:
    assert (entry == 0 or entry == 1), "heaving entries should be booleans."

left = 0
for radius in a:
    assert radius > left, "a entries should be increasing, and start greater than 0."
    left = radius

for depth in d:
    assert depth >= 0, "d entries should be nonnegative."
    assert depth < h, "d entries should be less than h."

for val in NMK:
    assert (type(val) == int and val > 0), "NMK entries should be positive integers."

assert (m0 > 0), "m0 should be positive." # currently shouldn't be too large, but that will be fixed eventually


In [ ]:
# CREATING THE A MATRIX
size = NMK[0] + NMK[-1] + 2 * sum(NMK[1:len(NMK) - 1])
boundary_count = len(NMK) - 1

rows = [] # collection of rows of blocks in A matrix, to be concatenated later

## Define values/functions to help block creation
#Coupling integral values
I_nm_vals = np.zeros((max(NMK), max(NMK), boundary_count - 1), dtype = complex)
for bd in range(boundary_count - 1):
    for n in range(NMK[bd]):
        for m in range(NMK[bd + 1]):
            I_nm_vals[n][m][bd] = I_nm(n, m, bd)
I_mk_vals = np.zeros((NMK[boundary_count - 1], NMK[boundary_count]), dtype = complex)
for m in range(NMK[boundary_count - 1]):
    for k in range(NMK[boundary_count]):
        I_mk_vals[m][k]= I_mk(m, k, boundary_count - 1)

## Functions to create blocks of certain types
# arguments: diagonal block on left (T/F), vectorized radial eigenfunction, boundary number
def p_diagonal_block(left, radfunction, bd):
    region = bd if left else (bd + 1)
    sign = 1 if left else (-1)
    return sign * (h - d[region]) * np.diag(radfunction(list(range(NMK[region])), a[bd], region))
    
# arguments: dense block on left (T/F), vectorized radial eigenfunction, boundary number
def p_dense_block(left, radfunction, bd):
    I_nm_array = I_nm_vals[0:NMK[bd],0:NMK[bd+1], bd]
    if left: # determine which is region to work in and which is adjacent
        region, adj = bd, bd + 1
        sign = 1
        I_nm_array = np.transpose(I_nm_array)
    else:
        region, adj = bd + 1, bd
        sign = -1
    radial_vector = radfunction(list(range(NMK[region])), a[bd], region)
    radial_array = np.outer((np.full((NMK[adj]), 1)), radial_vector)
    return sign * radial_array * I_nm_array

def p_dense_block_e(bd):
    I_mk_array = I_mk_vals
    radial_vector = (np.vectorize(Lambda_k, otypes = [complex]))(list(range(NMK[bd+1])), a[bd])
    radial_array = np.outer((np.full((NMK[bd]), 1)), radial_vector)
    return (-1) * radial_array * I_mk_array

#####
# arguments: diagonal block on left (T/F), vectorized radial eigenfunction, boundary number
def v_diagonal_block(left, radfunction, bd):
    region = bd if left else (bd + 1)
    sign = (-1) if left else (1)
    return sign * (h - d[region]) * np.diag(radfunction(list(range(NMK[region])), a[bd], region))

# arguments: dense block on left (T/F), vectorized radial eigenfunction, boundary number
def v_dense_block(left, radfunction, bd):
    I_nm_array = I_nm_vals[0:NMK[bd],0:NMK[bd+1], bd]
    if left: # determine which is region to work in and which is adjacent
        region, adj = bd, bd + 1
        sign = -1
        I_nm_array = np.transpose(I_nm_array)
    else:
        region, adj = bd + 1, bd
        sign = 1
    radial_vector = radfunction(list(range(NMK[region])), a[bd], region)
    radial_array = np.outer((np.full((NMK[adj]), 1)), radial_vector)
    return sign * radial_array * I_nm_array

def v_diagonal_block_e(bd):
    return h * np.diag((np.vectorize(diff_Lambda_k, otypes = [complex]))(list(range(NMK[bd+1])), a[bd]))

def v_dense_block_e(radfunction, bd): # for region adjacent to e-type region
    I_km_array = np.transpose(I_mk_vals)
    radial_vector = radfunction(list(range(NMK[bd])), a[bd], bd)
    radial_array = np.outer((np.full((NMK[bd + 1]), 1)), radial_vector)
    return (-1) * radial_array * I_km_array


In [ ]:
# Potential Blocks
col = 0
for bd in range(boundary_count):
    N = NMK[bd]
    M = NMK[bd + 1]
    if bd == (boundary_count - 1): # i-e boundary, inherently left diagonal
        row_height = N
        left_block1 = p_diagonal_block(True, np.vectorize(R_1n), bd)
        right_block = p_dense_block_e(bd)
        if bd == 0: # one cylinder
            rows.append(np.concatenate((left_block1,right_block), axis = 1))
        else:
            left_block2 = p_diagonal_block(True, np.vectorize(R_2n), bd)
            left_zeros = np.zeros((row_height, col), dtype=complex)
            rows.append(np.concatenate((left_zeros,left_block1,left_block2,right_block), axis = 1))
    elif bd == 0:
        left_diag = d[bd] > d[bd + 1] # which of the two regions gets diagonal entries
        if left_diag:
            row_height = N
            left_block = p_diagonal_block(True, np.vectorize(R_1n), 0)
            right_block1 = p_dense_block(False, np.vectorize(R_1n), 0)
            right_block2 = p_dense_block(False, np.vectorize(R_2n), 0)
        else:
            row_height = M
            left_block = p_dense_block(True, np.vectorize(R_1n), 0)
            right_block1 = p_diagonal_block(False, np.vectorize(R_1n), 0)
            right_block2 = p_diagonal_block(False, np.vectorize(R_2n), 0)
        right_zeros = np.zeros((row_height, size - (col + N + 2 * M)),dtype=complex)
        block_lst = [left_block, right_block1, right_block2, right_zeros]
        rows.append(np.concatenate(block_lst, axis = 1))
        col += N
    else: # i-i boundary
        left_diag = d[bd] > d[bd + 1] # which of the two regions gets diagonal entries
        if left_diag:
            row_height = N
            left_block1 = p_diagonal_block(True, np.vectorize(R_1n), bd)
            left_block2 = p_diagonal_block(True, np.vectorize(R_2n), bd)
            right_block1 = p_dense_block(False, np.vectorize(R_1n),  bd)
            right_block2 = p_dense_block(False, np.vectorize(R_2n),  bd)
        else:
            row_height = M
            left_block1 = p_dense_block(True, np.vectorize(R_1n),  bd)
            left_block2 = p_dense_block(True, np.vectorize(R_2n),  bd)
            right_block1 = p_diagonal_block(False, np.vectorize(R_1n),  bd)
            right_block2 = p_diagonal_block(False, np.vectorize(R_2n),  bd)
        left_zeros = np.zeros((row_height, col), dtype=complex)
        right_zeros = np.zeros((row_height, size - (col + 2 * N + 2 * M)),dtype=complex)
        block_lst = [left_zeros, left_block1, left_block2, right_block1, right_block2, right_zeros]
        rows.append(np.concatenate(block_lst, axis = 1))
        col += 2 * N


###############################
# Velocity Blocks
col = 0
for bd in range(boundary_count):
    N = NMK[bd]
    M = NMK[bd + 1]
    if bd == (boundary_count - 1): # i-e boundary, inherently left diagonal
        row_height = M
        left_block1 = v_dense_block_e(np.vectorize(diff_R_1n, otypes=[complex]), bd)
        right_block = v_diagonal_block_e(bd)
        if bd == 0: # one cylinder
            rows.append(np.concatenate((left_block1,right_block), axis = 1))
        else:
            left_block2 = v_dense_block_e(np.vectorize(diff_R_2n, otypes=[complex]), bd)
            left_zeros = np.zeros((row_height, col), dtype=complex)
            rows.append(np.concatenate((left_zeros,left_block1,left_block2,right_block), axis = 1))
    elif bd == 0:
        left_diag = d[bd] < d[bd + 1] # taller fluid region gets diagonal entries
        if left_diag:
            row_height = N
            left_block = v_diagonal_block(True, np.vectorize(diff_R_1n, otypes=[complex]), 0)
            right_block1 = v_dense_block(False, np.vectorize(diff_R_1n, otypes=[complex]), 0)
            right_block2 = v_dense_block(False, np.vectorize(diff_R_2n, otypes=[complex]), 0)
        else:
            row_height = M
            left_block = v_dense_block(True, np.vectorize(diff_R_1n, otypes=[complex]), 0)
            right_block1 = v_diagonal_block(False, np.vectorize(diff_R_1n, otypes=[complex]), 0)
            right_block2 = v_diagonal_block(False, np.vectorize(diff_R_2n, otypes=[complex]), 0)
        right_zeros = np.zeros((row_height, size - (col + N + 2 * M)),dtype=complex)
        block_lst = [left_block, right_block1, right_block2, right_zeros]
        rows.append(np.concatenate(block_lst, axis = 1))
        col += N
    else: # i-i boundary
        left_diag = d[bd] < d[bd + 1] # taller fluid region gets diagonal entries
        if left_diag:
            row_height = N
            left_block1 = v_diagonal_block(True, np.vectorize(diff_R_1n, otypes=[complex]), bd)
            left_block2 = v_diagonal_block(True, np.vectorize(diff_R_2n, otypes=[complex]), bd)
            right_block1 = v_dense_block(False, np.vectorize(diff_R_1n, otypes=[complex]),  bd)
            right_block2 = v_dense_block(False, np.vectorize(diff_R_2n, otypes=[complex]),  bd)
        else:
            row_height = M
            left_block1 = v_dense_block(True, np.vectorize(diff_R_1n, otypes=[complex]),  bd)
            left_block2 = v_dense_block(True, np.vectorize(diff_R_2n, otypes=[complex]),  bd)
            right_block1 = v_diagonal_block(False, np.vectorize(diff_R_1n, otypes=[complex]),  bd)
            right_block2 = v_diagonal_block(False, np.vectorize(diff_R_2n, otypes=[complex]),  bd)
        left_zeros = np.zeros((row_height, col), dtype=complex)
        right_zeros = np.zeros((row_height, size - (col + 2* N + 2 * M)),dtype=complex)
        block_lst = [left_zeros, left_block1, left_block2, right_block1, right_block2, right_zeros]
        rows.append(np.concatenate(block_lst, axis = 1))
        col += 2 * N

## Concatenate the rows of blocks into the square A matrix
A = np.concatenate(rows, axis = 0)

###########################################################################
# This plots a sparsity matrix
if False:
    
 rows, cols = np.nonzero(A)
 plt.figure(figsize=(6, 6))
 plt.scatter(cols, rows, color='blue', marker='o', s=100) 
 plt.gca().invert_yaxis() 
 plt.xticks(range(A.shape[1]))
 plt.yticks(range(A.shape[0]))

 cols = [NMK[0]]
 for i in range(1, boundary_count):
    cols.append(cols[-1] + NMK[i])
    cols.append(cols[-1] + NMK[i])
 cols.append(cols[-1] + NMK[-1])

 for val in cols:
    plt.axvline(val-0.5, color='black', linestyle='-', linewidth=1) 
    plt.axhline(val-0.5, color='black', linestyle='-', linewidth=1) 

 # for y in range(0, A.shape[0], 3):
    # plt.axhline(y-0.5, color='black', linestyle='-', linewidth=1) 

 plt.grid(True)
 plt.title('Non-Zero Entries of the Matrix')
 plt.xlabel('Column Index')
 plt.ylabel('Row Index')
 plt.show()


In [ ]:
b = np.zeros(size, dtype=complex)

index = 0

# potential matching
for boundary in range(boundary_count):
    if boundary == (boundary_count - 1): # i-e boundary
        for n in range(NMK[-2]):
            b[index] = b_potential_end_entry(n, boundary)
            index += 1
    else: # i-i boundary
        for n in range(NMK[boundary + (d[boundary] < d[boundary + 1])]): # iterate over eigenfunctions for smaller h-d
            b[index] = b_potential_entry(n, boundary)
            index += 1

# velocity matching
for boundary in range(boundary_count):
    if boundary == (boundary_count - 1): # i-e boundary
        for n in range(NMK[-1]):
            b[index] = b_velocity_end_entry(n, boundary)
            index += 1
    else: # i-i boundary
        for n in range(NMK[boundary + (d[boundary] > d[boundary + 1])]): # iterate over eigenfunctions for larger h-d
            b[index] = b_velocity_entry(n, boundary)
            index += 1

In [ ]:
X = linalg.solve(A,b)

In [ ]:
# The c-vector dotted with X is the hydro coefficient integral (+ a constant, sum(hydro_p_terms))
c = np.zeros((size - NMK[-1]), dtype=complex)
col = 0
for n in range(NMK[0]):
    c[n] = heaving[0] * int_R_1n(0, n)* z_n_d(n)
col += NMK[0]
for i in range(1, boundary_count):
    M = NMK[i]
    for m in range(M):
        c[col + m] = heaving[i] * int_R_1n(i, m)* z_n_d(m)
        c[col + M + m] = heaving[i] * int_R_2n(i, m)* z_n_d(m)
    col += 2 * M

hydro_p_terms = np.zeros(boundary_count, dtype=complex)
for i in range(boundary_count):
    hydro_p_terms[i] = heaving[i] * int_phi_p_i(i)

hydro_coef = 2 * pi * (np.dot(c, X[:-NMK[-1]]) + sum(hydro_p_terms))
hydro_coef_real = hydro_coef.real * h**3 * rho
hydro_coef_imag = hydro_coef.imag * omega * h**3 * rho

# find maximum heaving radius
max_rad = a[0]
for i in range(boundary_count - 1, 0, -1):
    if heaving[i]:
        max_rad = a[i]
        break

hydro_coef_nondim = h**3/(max_rad**3 * pi)*hydro_coef

print("real (added mass):", hydro_coef_real)
print("imag (damping):", hydro_coef_imag)
print("real/(h^3):", hydro_coef_real/(h**3)) # to compare with Capytaine
print("imag/(h^3):", hydro_coef_imag/(h**3))
print("nondimensional, real:", hydro_coef_nondim.real)
print("nondimensional, imag (no omega factor):", hydro_coef_nondim.imag)

print("Excitation Phase: ", excitation_phase(X[-NMK[-1]]), " radians")

# print("ratio", hydro_coef_real/hydro_coef_imag)

In [ ]:
# Split up the Cs into groups depending on which equation they belong to.
Cs = []
row = 0
Cs.append(X[:NMK[0]])
row += NMK[0]
for i in range(1, boundary_count):
    Cs.append(X[row: row + NMK[i] * 2])
    row += NMK[i] * 2
Cs.append(X[row:])

In [ ]:
def make_R_Z(sharp, spatial_res): # create coordinate array for graphing
    rmin = (2 * a[-1] / spatial_res) if sharp else 0.0
    r_vec = np.linspace(rmin, 2*a[-1], spatial_res)
    z_vec = np.linspace(0, -h, spatial_res)
    if sharp: # more precise near boundaries
        a_eps = 1.0e-4
        for i in range(len(a)):
            r_vec = np.append(r_vec, a[i]*(1-a_eps))
            r_vec = np.append(r_vec, a[i]*(1+a_eps))
        r_vec = np.unique(r_vec)
        for i in range(len(d)):
            z_vec = np.append(z_vec, -d[i])
        z_vec = np.unique(z_vec)
    return np.meshgrid(r_vec, z_vec)


In [ ]:
def phi_h_n_inner_func(n, r, z):
    return (Cs[0][n] * R_1n(n, r, 0)) * Z_n_i(n, z, 0)

def phi_h_m_i_func(i, m, r, z):
    return (Cs[i][m] * R_1n(m, r, i) + Cs[i][NMK[i] + m] * R_2n(m, r, i)) * Z_n_i(m, z, i)

def phi_e_k_func(k, r, z):
    return Cs[-1][k] * Lambda_k(k, r) * Z_k_e(k, z)

phi_e_k_vec = np.vectorize(phi_e_k_func, otypes = [complex])
phi_h_n_inner_vec = np.vectorize(phi_h_n_inner_func, otypes = [complex])
phi_h_m_i_vec = np.vectorize(phi_h_m_i_func, otypes = [complex])


R, Z = make_R_Z(True, 50)

regions = []
regions.append((R <= a[0]) & (Z < -d[0]))
for i in range(1, boundary_count):
    regions.append((R > a[i-1]) & (R <= a[i]) & (Z < -d[i]))
regions.append(R > a[-1])

phi = np.full_like(R, np.nan + np.nan*1j, dtype=complex) 
phiH = np.full_like(R, np.nan + np.nan*1j, dtype=complex) 
phiP = np.full_like(R, np.nan + np.nan*1j, dtype=complex) 

for n in range(NMK[0]):
    temp_phiH = phi_h_n_inner_vec(n, R[regions[0]], Z[regions[0]])
    phiH[regions[0]] = temp_phiH if n == 0 else phiH[regions[0]] + temp_phiH

for i in range(1, boundary_count):
    for m in range(NMK[i]):
        temp_phiH = phi_h_m_i_vec(i, m, R[regions[i]], Z[regions[i]])
        phiH[regions[i]] = temp_phiH if m == 0 else phiH[regions[i]] + temp_phiH

for k in range(NMK[-1]):
    temp_phiH = phi_e_k_vec(k, R[regions[-1]], Z[regions[-1]])
    phiH[regions[-1]] = temp_phiH if k == 0 else phiH[regions[-1]] + temp_phiH

phi_p_i_vec = np.vectorize(phi_p_i)

phiP[regions[0]] = heaving[0] * phi_p_i_vec(d[0], R[regions[0]], Z[regions[0]])
for i in range(1, boundary_count):
    phiP[regions[i]] = heaving[i] * phi_p_i_vec(d[i], R[regions[i]], Z[regions[i]])
phiP[regions[-1]] = 0

phi = phiH + phiP

#nanregions = []
#nanregions.append((R <= a[0]) & (Z > -d[0]))
#for i in range(1, len(a)):
#    nanregions.append((R > a[i-1]) & (R <= a[i]) & (Z > -d[i]))


In [ ]:
def plot_potential(field, R, Z, title):
    plt.figure(figsize=(8, 6))
    plt.contourf(R, Z, field, levels=50, cmap='viridis')
    plt.colorbar()
    plt.title(title)
    plt.xlabel('Radial Distance (R)')
    plt.ylabel('Axial Distance (Z)')
    plt.show()

plot_potential(np.real(phiH), R, Z, 'Homogeneous Potential')
plot_potential(np.imag(phiH), R, Z, 'Homogeneous Potential Imaginary')

plot_potential(np.real(phiP), R, Z, 'Particular Potential')
plot_potential(np.imag(phiP), R, Z, 'Particular Potential Imaginary')

plot_potential(np.real(phi), R, Z, 'Potential (Real Part)')
plot_potential(np.imag(phi), R, Z, 'Total Potential Imaginary')




In [ ]:
def v_r_inner_func(n, r, z):
    return (Cs[0][n] * diff_R_1n(n, r, 0)) * Z_n_i(n, z, 0)

def v_r_m_i_func(i, m, r, z):
    return (Cs[i][m] * diff_R_1n(m, r, i) + Cs[i][NMK[i] + m] * diff_R_2n(m, r, i)) * Z_n_i(m, z, i)

def v_r_e_k_func(k, r, z):
    return Cs[-1][k] * diff_Lambda_k(k, r) * Z_k_e(k, z)

def v_z_inner_func(n, r, z):
    return (Cs[0][n] * R_1n(n, r, 0)) * diff_Z_n_i(n, z, 0)

def v_z_m_i_func(i, m, r, z):
    return (Cs[i][m] * R_1n(m, r, i) + Cs[i][NMK[i] + m] * R_2n(m, r, i)) * diff_Z_n_i(m, z, i)

def v_z_e_k_func(k, r, z):
    return Cs[-1][k] * Lambda_k(k, r) * diff_Z_k_e(k, z)

v_r_inner_vec = np.vectorize(v_r_inner_func, otypes = [complex])
v_r_m_i_vec = np.vectorize(v_r_m_i_func, otypes = [complex])
v_r_e_k_vec = np.vectorize(v_r_e_k_func, otypes = [complex])
v_z_inner_vec = np.vectorize(v_z_inner_func, otypes = [complex])
v_z_m_i_vec = np.vectorize(v_z_m_i_func, otypes = [complex])
v_z_e_k_vec = np.vectorize(v_z_e_k_func, otypes = [complex])

In [ ]:
vr = np.full_like(R, np.nan + np.nan*1j, dtype=complex) 
vrH = np.full_like(R, np.nan + np.nan*1j, dtype=complex) 
vrP = np.full_like(R, np.nan + np.nan*1j, dtype=complex) 

vz = np.full_like(R, np.nan + np.nan*1j, dtype=complex) 
vzH = np.full_like(R, np.nan + np.nan*1j, dtype=complex) 
vzP = np.full_like(R, np.nan + np.nan*1j, dtype=complex)

for n in range(NMK[0]):
    temp_vrH = v_r_inner_vec(n, R[regions[0]], Z[regions[0]])
    temp_vzH = v_z_inner_vec(n, R[regions[0]], Z[regions[0]])
    if n == 0:
        vrH[regions[0]] = temp_vrH
        vzH[regions[0]] = temp_vzH
    else:
        vrH[regions[0]] = vrH[regions[0]] + temp_vrH
        vzH[regions[0]] = vzH[regions[0]] + temp_vzH

for i in range(1, boundary_count):
    for m in range(NMK[i]):
        temp_vrH = v_r_m_i_vec(i, m, R[regions[i]], Z[regions[i]])
        temp_vzH = v_z_m_i_vec(i, m, R[regions[i]], Z[regions[i]])
        if m == 0:
            vrH[regions[i]] = temp_vrH
            vzH[regions[i]] = temp_vzH
        else:
            vrH[regions[i]] = vrH[regions[i]] + temp_vrH
            vzH[regions[i]] = vzH[regions[i]] + temp_vzH

for k in range(NMK[-1]):
    temp_vrH = v_r_e_k_vec(k, R[regions[-1]], Z[regions[-1]])
    temp_vzH = v_z_e_k_vec(k, R[regions[-1]], Z[regions[-1]])
    if k == 0:
        vrH[regions[-1]] = temp_vrH
        vzH[regions[-1]] = temp_vzH
    else:
        vrH[regions[-1]] = vrH[regions[-1]] + temp_vrH
        vzH[regions[-1]] = vzH[regions[-1]] + temp_vzH

vr_p_i_vec = np.vectorize(diff_r_phi_p_i)
vz_p_i_vec = np.vectorize(diff_z_phi_p_i)

vrP[regions[0]] = heaving[0] * vr_p_i_vec(d[0], R[regions[0]], Z[regions[0]])
vzP[regions[0]] = heaving[0] * vz_p_i_vec(d[0], R[regions[0]], Z[regions[0]])
for i in range(1, boundary_count):
    vrP[regions[i]] = heaving[i] * vr_p_i_vec(d[i], R[regions[i]], Z[regions[i]])
    vzP[regions[i]] = heaving[i] * vz_p_i_vec(d[i], R[regions[i]], Z[regions[i]])
vrP[regions[-1]] = 0
vzP[regions[-1]] = 0

vr = vrH + vrP
vz = vzH + vzP

In [ ]:
plot_potential(np.real(vr), R, Z, 'Radial Velocity - Real')
plot_potential(np.imag(vr), R, Z, 'Radial Velocity - Imaginary')
plot_potential(np.real(vz), R, Z, 'Vertical Velocity - Real')
plot_potential(np.imag(vz), R, Z, 'Vertical Velocity - Imaginary')

In [ ]:
# Format Potential Matrix for Testing:
R, Z = make_R_Z(False, 50)

regions = []
regions.append((R <= a[0]) & (Z < -d[0]))
for i in range(1, boundary_count):
    regions.append((R > a[i-1]) & (R <= a[i]) & (Z < -d[i]))
regions.append(R > a[-1])

phi = np.full_like(R, np.nan + np.nan*1j, dtype=complex) 
phiH = np.full_like(R, np.nan + np.nan*1j, dtype=complex) 
phiP = np.full_like(R, np.nan + np.nan*1j, dtype=complex) 

for n in range(NMK[0]):
    temp_phiH = phi_h_n_inner_vec(n, R[regions[0]], Z[regions[0]])
    phiH[regions[0]] = temp_phiH if n == 0 else phiH[regions[0]] + temp_phiH

for i in range(1, boundary_count):
    for m in range(NMK[i]):
        temp_phiH = phi_h_m_i_vec(i, m, R[regions[i]], Z[regions[i]])
        phiH[regions[i]] = temp_phiH if m == 0 else phiH[regions[i]] + temp_phiH

for k in range(NMK[-1]):
    temp_phiH = phi_e_k_vec(k, R[regions[-1]], Z[regions[-1]])
    phiH[regions[-1]] = temp_phiH if k == 0 else phiH[regions[-1]] + temp_phiH

phi_p_i_vec = np.vectorize(phi_p_i)

phiP[regions[0]] = heaving[0] * phi_p_i_vec(d[0], R[regions[0]], Z[regions[0]])
for i in range(1, boundary_count):
    phiP[regions[i]] = heaving[i] * phi_p_i_vec(d[i], R[regions[i]], Z[regions[i]])
phiP[regions[-1]] = 0

phi = phiH + phiP

nanregions = []
nanregions.append((R <= a[0]) & (Z > -d[0]))
for i in range(1, len(a)):
    nanregions.append((R > a[i-1]) & (R <= a[i]) & (Z > -d[i]))

In [ ]:
def interpret_file(filename, omega): # comparison with Capytaine
    file_path = "test/data/" + filename + "-imag.csv"
    df = (pd.read_csv(file_path, header=None)).transpose()
    real_array = (df.to_numpy()) * (-1/omega)
        
    file_path = "test/data/" + filename + "-real.csv"
    df = (pd.read_csv(file_path, header=None)).transpose()
    imag_array = (df.to_numpy()) * (1/omega)

    return(real_array, imag_array)

def interpret_file2(filename):
    file_path = "test/data/" + filename + "-imag - matlab.csv"
    df = (pd.read_csv(file_path, header=None))
    #df_flipped = df.iloc[::-1].reset_index(drop=True)
    imag_array = df.to_numpy()
        
    file_path = "test/data/" + filename + "-real - matlab.csv"
    df = (pd.read_csv(file_path, header=None))
    #df_flipped = df.iloc[::-1].reset_index(drop=True)
    real_array = df.to_numpy()

    return(real_array, imag_array)

def populate_column(prefer_lower, l_col, h_col, smooth):
    new_col = np.empty_like(l_col)
    if smooth:
        for i in range(len(l_col)):
            if np.isnan(l_col[i]) or np.isnan(h_col[i]):
                if prefer_lower:
                    new_col[i] = l_col[i]
                else:
                    new_col[i] = h_col[i]
            else:
                new_col[i] = ((l_col[i] + h_col[i]) / 2)
    else:
        if prefer_lower:
            new_col = l_col
        else:
            new_col = h_col
    return new_col.reshape(-1, 1)

def sharpen(arr, smooth):
    # just for plotting purposes, add some points near the boundary that take on value of nearby points for sharp corners
    r_vec = np.linspace(0, 2*a[-1], 50)
    z_vec = np.linspace(0, -h, 50)
    a_eps = 1.0e-4
    for i in range(len(a)):
        lower = a[i]*(1-a_eps)
        idx = np.searchsorted(r_vec, lower)
        l_col = arr[:, idx - 1]
        h_col = arr[:, idx]
        if lower not in r_vec:
            r_vec = np.insert(r_vec, idx, lower)
            new_col = populate_column(True, l_col, h_col, smooth)
            arr = np.hstack([arr[:, :idx], new_col, arr[:, idx:]])
        higher = a[i]*(1+a_eps)
        if higher not in r_vec:
            idx = np.searchsorted(r_vec, higher)
            r_vec = np.insert(r_vec, idx, higher)
            new_col = populate_column(False, l_col, h_col, smooth)
            arr = np.hstack([arr[:, :(idx + 1)], new_col, arr[:, (idx + 1):]])
    return np.meshgrid(r_vec, z_vec), arr

def plot_comparison(field, title, color):
    (R_new, Z_new), field = sharpen(field, True)
    plt.figure(figsize=(8, 6))
    plt.contourf(R_new, Z_new, field, levels=50, cmap = color)
    plt.colorbar()
    plt.title(title)
    plt.xlabel('Radial Distance (R)')
    plt.ylabel('Axial Distance (Z)')
    plt.show()

def plot_difference(title, R, Z, arr1, arr2):
    arr3 = arr1 + arr2
    (R_new, Z_new), field = sharpen(arr3, False)
    plt.figure(figsize=(8, 6))
    plt.contourf(R_new, Z_new, field, levels=2, cmap='viridis')
    plt.colorbar()
    plt.title(title)
    plt.xlabel('Radial Distance (R)')
    plt.ylabel('Axial Distance (Z)')
    plt.show()

# arguments: the name of the csv to compare with, an appropriately formatted potential array
# the threshold of closeness, appropriate R and Z, and omega to help with conversion
# tailored for 50x50 points (including nans), evenly spaced, twice the widest radius, and given height

def potential_comparison(filename, arr, threshold, thres2, R, Z, omega, nan_mask):

    real_arr, imag_arr = interpret_file(filename, omega)
    real_calc_arr = np.real(arr)
    imag_calc_arr = np.imag(arr)

    plot_comparison(real_arr, 'Capytaine Potential Real', "viridis")
    plot_comparison(real_calc_arr, 'MEEM Potential Real', "viridis")
    plot_comparison(imag_arr, 'Capytaine Potential Imaginary', "viridis")
    plot_comparison(imag_calc_arr, 'MEEM Potential Imaginary', "viridis")

    plot_comparison(real_arr - real_calc_arr, 'Real Potential Difference', "plasma")
    plot_comparison(imag_arr - imag_calc_arr, 'Imag Potential Difference', "plasma")

    raw_fraction_real = (real_arr - real_calc_arr)/real_arr
    raw_fraction_imag = (imag_arr - imag_calc_arr)/imag_arr

    fraction_real = np.where(abs(raw_fraction_real) < 0.1, raw_fraction_real, np.nan)
    fraction_imag = np.where(abs(raw_fraction_imag) < 1, raw_fraction_imag, np.nan)

    plot_comparison(fraction_real, 'Fractional Real Potential Difference', "plasma")
    plot_comparison(fraction_imag, 'Fractional Imag Potential Difference', "plasma")

    is_within_threshold_r = 1. * np.isclose(real_arr, np.real(arr), rtol=threshold, atol = 0.01)
    is_within_threshold_i = 1. * np.isclose(imag_arr, np.imag(arr), rtol=threshold, atol = 0.01)

    iwt_r_2 = 1. * np.isclose(real_arr, np.real(arr), rtol=thres2, atol = 0.01)
    iwt_i_2 = 1. * np.isclose(imag_arr, np.imag(arr), rtol=thres2, atol = 0.01)

    for i in range(len(nan_mask)):
        is_within_threshold_r[nan_mask[i]] = np.nan
        is_within_threshold_i[nan_mask[i]] = np.nan
        iwt_r_2[nan_mask[i]] = np.nan
        iwt_i_2[nan_mask[i]] = np.nan
        
    plot_difference("Real Potential Match", R, Z, is_within_threshold_r, iwt_r_2) #, " + str(100 * threshold) + "%"
    plot_difference("Imaginary Match", R, Z, is_within_threshold_i, iwt_i_2)

    match_r = np.sum(np.isnan(is_within_threshold_r)) + np.sum(is_within_threshold_r == 1)
    match_i = np.sum(np.isnan(is_within_threshold_i)) + np.sum(is_within_threshold_i == 1)

    return (match_r, match_i)



In [ ]:
potential_comparison("config6", phi, 0.01, 0.03, R, Z, omega, nanregions)

In [ ]:
# Run this code block for a quick visual of our results converted to the Capytaine conventions.
# Capytaine should output something that is off by a factor of (-) i * omega. The following graphs do that:
# Figure out why vrc factor is different from phic and vrz (no negative sign) (likely due to times i vs divide by i)

#phic = - phi * 1j * omega
#vrc = vr * 1j * omega
#vzc = - vz * 1j * omega

#R, Z = make_R_Z(False, 50) # phi was redefined into this coordinate array earlier
#plot_potential(np.real(phic), R, Z, 'Capytaine Potential Real')
#plot_potential(np.imag(phic), R, Z, 'Total Potential Imaginary')

#R, Z = make_R_Z(True, 50) # "sharp" coordinate array
#plot_potential(np.real(vrc), R, Z, 'Radial Velocity - Real') 
#plot_potential(np.imag(vrc), R, Z, 'Radial Velocity - Imaginary')
#plot_potential(np.real(vzc), R, Z, 'Vertical Velocity - Real')
#plot_potential(np.imag(vzc), R, Z, 'Vertical Velocity - Imaginary')


In [ ]:
# Refer to original MEEM file for some extra plotting functions.